# TD Ameritrade API  
## Authentication  
https://developer.tdameritrade.com/apis  
https://github.com/alexgolec/tda-api/tree/b4e9433ee19927f6a8d1b397bcc6099d4abf9c94

Install modules

In [10]:
import sys
# !{sys.executable} -m pip install 

Load modules

In [1]:
import os
import requests
import json
import time
from datetime import datetime, timedelta
import urllib
from selenium import webdriver
from authlib.integrations.requests_client import OAuth2Session, OAuth2Auth
import websockets.legacy.client as ws_client
import inspect
import asyncio
from enum import Enum
from collections import defaultdict, deque
import copy

Set project base path

In [4]:
base_path = os.path.join(os.path.expanduser('~'), 'bakery')
creds_path = os.path.join(base_path, 'td_ameritrade/creds.json')
token_path = os.path.join(base_path, 'td_ameritrade/tokens.json')
print(base_path)

/home/mark/bakery


Load credential file

In [5]:
with open(creds_path, 'rb') as f:
    data = f.read()
    creds = json.loads(data.decode())

In [6]:
creds

{'username': 'markeidsaune',
 'password': 'XeZtMV6q4Uof',
 'dev_username': 'markeidsaune@gmail.com',
 'dev_password': 'cV*H11&y3Rrm',
 'api_key': 'FMNMNUOPZ1FMWCJ4CH1EPQONALLUYM9T',
 'redirect_uri': 'https://localhost',
 'account_no': '455146202'}

##### Get token  
Webdriver workflow

Create OAuth2 client

In [ ]:
client_id = creds['api_key'] + '@AMER.OAUTHAP'
client_secret = creds['password']
token_endpoint = 'https://api.tdameritrade.com/v1/oauth2/token'
redirect_uri = creds['redirect_uri']

client = OAuth2Session(
    client_id=client_id, 
    client_secret=client_secret,
    token_endpoint=token_endpoint,
    redirect_uri=redirect_uri
)

Get authorization url

In [ ]:
uri, state = client.create_authorization_url(
    url='https://auth.tdameritrade.com/auth'
)

Create browser object

In [ ]:
driver = webdriver.Firefox()
driver.get(uri)

Wait until current url starts with the redirect_url

In [ ]:
current_url = ''
num_waits = 0
max_waits = 3000
wait_time = 0.1

while not current_url.startswith(redirect_uri):
    current_url = driver.current_url
    
    if num_waits > max_waits:
        raise RedirectTimeoutError('Timed out waiting for redirect')
    
    time.sleep(wait_time)
    
    num_waits += 1

Call the fetch_token method w/ access_type='offline' to get an access token and refresh token

In [ ]:
token = client.fetch_token(
    authorization_response=current_url,
    access_type='offline'
)

In [ ]:
with open(token_path, 'w') as f:
    json.dump(token, f)

In [ ]:
print(
    'Token expires in {} minutes'.format(
        token['expires_in'] / 60
    )
)
print(
    'Token expires @ {}'.format(
        datetime.fromtimestamp(token['expires_at'])
    )
)

In [ ]:
print(
    'Refresh token expires in {} days'.format(
        token['refresh_token_expires_in'] / 60 / 60 / 24
    )
)
print(
    'Refresh token expires @ {}'.format(
    datetime.fromtimestamp(token['expires_at']) - timedelta(minutes=30) + timedelta(days=90)
    )
)

##### Get token  
with refresh token

Read current token file

with open(token_path, 'rb') as f:
    data = f.read()
    token = json.loads(data.decode())

In [6]:
current_dttm = datetime.now()

is_token_expired = (datetime.fromtimestamp(token['expires_at'])) \
    < current_dttm
is_refresh_token_expired = (datetime.fromtimestamp(token['expires_at']) \
        - timedelta(seconds=token['expires_in']) \
        + timedelta(seconds=token['refresh_token_expires_in'])) \
    < current_dttm

print('Token expired? {}\tRefresh token expired? {}'.format(
    is_token_expired, is_refresh_token_expired
    )
)

Token expired? True	Refresh token expired? False


In [7]:
client_id = creds['api_key'] + '@AMER.OAUTHAP'
client_secret = creds['password']
token_endpoint = 'https://api.tdameritrade.com/v1/oauth2/token'
redirect_uri = creds['redirect_uri']
refresh_token = token['refresh_token']

client = OAuth2Session(
    client_id=client_id, 
    client_secret=client_secret,
    token_endpoint=token_endpoint,
    redirect_uri=redirect_uri
)

In [8]:
new_token = client.fetch_token(
    token_endpoint,
    grant_type='refresh_token',
    refresh_token=token['refresh_token'],
    access_type='offline'
)
new_token

{'access_token': 'ZVGmXFCC4tIGUaq/dlD2aemktXb9k+SQtO5S7WQIQlFysY5GqdO18w/Mras0kc4DbV94+z4doQWYQPRggSJEsYzGglmXG2nTi3Jd0shq9oMkajAPdAFwRDiFkOHF9LjTeGGPMdZOZvhk9Qv8eDiULjXY1xy8AJv8qqMVrDPQhD5AqWmMLdxkGW2Yk38lr7zLUHHVvkLhFMvLZ32TG15mxyRHHiyQUOgeyfX7nfQrHfc4IX6ogemVCYHSPHDS9RFYqBWsyBzW8SeKqYlXwhcrCzZCoXrQ3hi3EPH3KHBX+/tNU4sjvjLf5XU5FhP6KA0fb/6mIOIJHWtN6R28Gh5b+rSq1Xz8i2WONw4eMazIhj17surncJiRWsI/RO9SXjbkCKIru2/Gngaf9bPHaE95Rnee3BAgItEjK9Tk1ncnKk6K4gkOEJzccQejvOMSxtaR0MLx1RZG13mjfAKaKsd0U4w6supAykg6n6W4qnZIGdNNZHqkD3livHhDttzf6VOrBEI9MALjNHn0WbHGvzZUdh0+bEV+1lxvgShtKXvnoNhNOxjKy100MQuG4LYrgoVi/JHHvl70XjMQBtJaMVowutMSp6DpcJASzy2E5eSqg6Rm+xNcNAwqVlSANmFBW9U0E8zUs+AzK7gTsBxcgO9PE/j9mJgRK3rhRKMEzVQPKmLmYxytGl5dAhkXVFUfuUirQVUdldFlvz5wdLiVpPtJxC4vll9hZBPOC4Ht2f7P2Gstbo+H6eLphGpP9ewUhxAUDDEAAJq2y2y18ixmKiMxnX1YQ7ZI6UaWyb7wV7mT8rUP4Vk7Nq1aMwuY0DG0HM/1ib9Rqd4eCr4eB+DOhQ2pTgWECYWZI8CkUxruZ8FN2g+9KjDglwUXx6DBThHVTGpcO/Tn9aIurXfItQ6cRCaVUEHlJrmMSnZHEvyAcixtZijk4h44gUqXh9xCWwnefOhKY52QH6hH+xzE6JrJu62OcJ

In [9]:
with open(token_path, 'w') as f:
    json.dump(new_token, f)

In [10]:
with open(token_path, 'rb') as f:
    data = f.read()
    token = json.loads(data.decode())

##### Get user principles

In [11]:
auth = OAuth2Auth(token)
payload = {'fields': 'streamerSubscriptionKeys,streamerConnectionInfo,preferences'}
r = requests.get(
    'https://api.tdameritrade.com/v1/userprincipals',
    auth=auth,
    params=payload
)

In [12]:
principals = r.json()